
## Project: Exporting and Enriching macOS Stickies Metadata

### Goal:
The purpose of this project is to create a Python script that:
1. Extracts notes from macOS Stickies.
2. Enriches the notes with metadata using an LLM agent.
3. Exports the enriched data to a JSON format suitable for indexing into a FAISS database.

### Use Case:
The enriched data will serve as a contextual layer for an LLM agent to provide communication advice. Specifically, the notes focus on follow-ups with individuals having different working styles.

### Workflow:
1. **Extract Notes**: Read the Stickies database and retrieve the content of each note.
2. **Generate Metadata**: Use an LLM agent to enrich the extracted content with metadata such as:
   - Tags
   - Priority
   - Related People
   - Communication Style
   - Contextual Information
3. **Export**: Save the enriched data to a structured JSON file for further use.



In [ ]:
!pip install striprtf


In [1]:
import os
import json
from pathlib import Path
from striprtf.striprtf import rtf_to_text
from datetime import datetime

def parse_rtf_content(rtf_string):
    """
    Parse the RTF content string to extract plain text.
    """
    try:
        return rtf_to_text(rtf_string)
    except Exception as e:
        print(f"Error parsing RTF content: {e}")
        return ""

def extract_text_from_rtfd(rtfd_path):
    """
    Extract text content from an RTFD folder by reading its RTF file.
    """
    main_rtf_file = Path(rtfd_path) / "TXT.rtf"
    if main_rtf_file.exists():
        try:
            with open(main_rtf_file, 'r', encoding='utf-8') as file:
                rtf_content = file.read()
                return parse_rtf_content(rtf_content)
        except Exception as e:
            print(f"Error reading {main_rtf_file}: {e}")
    return ""

def get_file_creation_time(file_path):
    """
    Get creation time for a file or directory.
    """
    try:
        stat_info = os.stat(file_path)
        created_at = datetime.fromtimestamp(stat_info.st_ctime).isoformat()
        return created_at
    except Exception as e:
        print(f"Error retrieving creation time for {file_path}: {e}")
        return None

def export_stickies_to_json(output_file="stickies_export.json"):
    """
    Collect all Stickies data from the Stickies folder and append them to a JSON file,
    ensuring no duplicates.
    """
    stickies_dir = Path("/Users/reneluijk/Library/Containers/com.apple.Stickies/Data/Library/Stickies")
    if not stickies_dir.exists():
        raise FileNotFoundError(f"Stickies folder not found at {stickies_dir}")
    
    # Load existing entries if the JSON file exists
    if Path(output_file).exists():
        with open(output_file, 'r', encoding='utf-8') as f:
            existing_notes = json.load(f)
    else:
        existing_notes = []
    
    # Create a set of existing note identifiers (filename + created_at) to check duplicates
    existing_ids = {(note["filename"], note["created_at"]) for note in existing_notes}

    # Iterate over each RTFD file in the Stickies directory
    new_notes = []
    for rtfd in stickies_dir.glob("*.rtfd"):
        note_content = extract_text_from_rtfd(rtfd)
        created_at = get_file_creation_time(rtfd)
        
        if note_content and (rtfd.stem, created_at) not in existing_ids:
            new_note = {
                "filename": rtfd.stem,
                "content": note_content.strip(),
                "created_at": created_at,
            }
            new_notes.append(new_note)
    
    # Append new notes to existing ones
    all_notes = existing_notes + new_notes
    
    # Save the updated list to JSON
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(all_notes, f, indent=4)
    
    print(f"Stickies exported successfully to {output_file}")

# Call the function
export_stickies_to_json()


Stickies exported successfully to stickies_export.json


In [ ]:
!pip install watchdog

In [ ]:
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import time
from pathlib import Path

class StickiesWatcher(FileSystemEventHandler):
    """
    Watches the Stickies folder for changes and triggers the export script.
    """
    def __init__(self, target_folder):
        self.target_folder = target_folder

    def on_any_event(self, event):
        # Trigger the script on any change (create, modify, delete)
        if event.is_directory:
            return  # Ignore folder-level changes
        print(f"Change detected: {event.src_path}")
        export_stickies_to_json()

# Setup and start the observer
def watch_stickies_folder(stickies_folder):
    stickies_path = Path(stickies_folder)
    if not stickies_path.exists():
        raise FileNotFoundError(f"Stickies folder not found at {stickies_folder}")

    event_handler = StickiesWatcher(stickies_folder)
    observer = Observer()
    observer.schedule(event_handler, path=stickies_folder, recursive=True)

    print(f"Watching for changes in {stickies_folder}...")
    observer.start()
    try:
        while True:
            time.sleep(1)  # Keep the script running
    except KeyboardInterrupt:
        print("Stopping watcher...")
        observer.stop()
    observer.join()

# Run the watcher
watch_stickies_folder("/Users/reneluijk/Library/Containers/com.apple.Stickies/Data/Library/Stickies")


# Continue

https://chatgpt.com/share/674e7b14-1e08-8006-8075-efcb23a53738


Here’s a list of the major decisions we need to make to proceed effectively:

---

### **1. Metadata and Index Structure**
- **What metadata to include**: 
  - Tags, timestamps, change types, sentiment, completion trends, etc.
  - Define which fields are essential for tracking and querying updates.
- **How to handle versioning**:
  - Store all versions of the sticky note with unique IDs.
  - Use `filename` + `version` or timestamps to link related versions.

---

### **2. Update Triggers**
- **When to update the FAISS index**:
  - On every detected change (with debouncing).
  - After significant content changes or periodic batching.
- **How to identify changes**:
  - Compare note content or rely on external indicators (e.g., timestamps).

---

### **3. Query Mechanism for the Agent**
- **How the agent queries the FAISS index**:
  - Query for historical versions of the same note.
  - Query for notes with similar tags or content.
  - Retrieve related notes for the same person or topic.
- **What results the agent expects**:
  - Historical context (e.g., past edits, trends).
  - Recommendations (e.g., communication patterns).

---

### **4. Agent Metadata Generation**
- **What insights the agent should produce**:
  - Tags, priorities, sentiment, completion patterns, etc.
  - Relational metadata (e.g., related notes, connections to individuals).
- **How the agent processes FAISS results**:
  - Use historical data to enrich the metadata.
  - Generate actionable recommendations (e.g., follow-up timing).

